# Behavior Cloning

### Data Augmentation and Formatting
Flip images and measurements

In [11]:
import csv
import cv2
import numpy as np

In [12]:
lines = []
with open("./data/driving_log.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        filename = line[0].split('/')
        try:
            filename = '/'.join(['data',filename[-2],filename[-1]])
        except:
            print(filename, "::", line)
        lines.append([filename, *line[-4:]])

fw = open("./data/driving_log_updated.csv", "w")
for line in lines:
    fw.write("{},{},{},{},{}\n".format(*line))
    if abs(float(line[1])) > 0.3:
        new_filename = line[0][:-4]+"_flipped.jpg"
        img = cv2.imread(line[0], cv2.IMREAD_COLOR)
        img = np.fliplr(img)
        fw.write("{},{},{},{},{}\n".format(new_filename, -1.0 * float(line[1]), line[2], line[3], line[4]))
        cv2.imwrite(new_filename, img)
    
fw.close()

Crop Images

In [ ]:
# 55 off top
# 30 off bottom
#fw = open("./data/driving_log_updated_cropped.csv", "w")
#with open("./data/driving_log_updated.csv") as f:
#    reader = csv.reader(f)
#    for line in reader:
#        filename = line[0]
#        img = cv2.imread(filename)
#        img = img[55:-30, :]
#        f_split = filename.split("/")
#        f_split[-2] = "CROPPED_IMG"
#        new_filename = '/'.join(f_split)
#        fw.write("{},{},{},{},{}\n".format(new_filename, *line[1:]))
#        cv2.imwrite(new_filename, img)
#    
#fw.close()

### Define Datasets

In [13]:
import random
import csv

TEST_SPLIT_RATIO = 0.2
VALIDATION_SPLIT_RATIO = 0.2

train_data = []
test_data = []
validation_data = []

with open("./data/driving_log_updated.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        rand = random.random()
        if rand < TEST_SPLIT_RATIO:
            test_data.append(line)
        elif rand < VALIDATION_SPLIT_RATIO+TEST_SPLIT_RATIO:
            validation_data.append(line)
        else:
            train_data.append(line)


In [14]:
total = 1.0* len(train_data)+len(test_data)+len(validation_data)
print(len(train_data), len(train_data)/total )
print(len(test_data), len(test_data)/total)
print(len(validation_data), len(validation_data)/total)


13758 0.5983299991302079
4635 0.20157432373662695
4601 0.20009567713316517


In [35]:
import sklearn
import random

def generator(samples, batch_size=32):
    num_samples = len(samples)
    samples = sklearn.utils.shuffle(samples)
    offset = 0
    while 1: # Loop forever so the generator never terminates
                    
        if offset >= len(samples):
            offset = 0
        vals = np.zeros((batch_size, 1))
        imgs = np.zeros((batch_size, 160, 320, 3))
        index = 0
        while index < batch_size:
            batch_sample = samples[offset]
            offset += 1
            if offset >= len(samples):
                offset = 0
            if abs(float(batch_sample[1])) < .1 and random.random() < .5:
                continue
            filename = batch_sample[0]
            center_image = cv2.imread(filename)
            imgs[index] = center_image
            try:
                float(batch_sample[1])
            except:
                print(batch_sample[1])
            vals[index][0] = float(batch_sample[1])
            index += 1

        # trim image to only see section with road
        X_train = imgs
        y_train = vals
            
        yield X_train, y_train


### Define Hyperparameters

In [36]:
epochs = 80
batch_size = 64

train_data_size = len(train_data)
test_data_size = len(test_data)
validation_data_size = len(validation_data)

### Define Model and Train

In [37]:
train_generator = generator(train_data, batch_size=batch_size)
validation_generator = generator(validation_data, batch_size=batch_size)
test_generator = generator(test_data, batch_size=batch_size)

In [39]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda, Flatten, Dropout
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.layers.advanced_activations import ELU
import keras.regularizers as regularizers

model = Sequential()
model.add(Cropping2D(cropping=((55,30),(0,0)), input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(75, 320, 3)))
model.add(Conv2D(16, (5,5), strides=(4, 4)))
model.add(ELU())
model.add(Conv2D(32, (5,5), strides=(2, 2)))
model.add(ELU())
model.add(Conv2D(64, (3,3), strides=(2, 2)))
model.add(ELU())
model.add(Conv2D(64, (3,3), strides=(2, 2)))
model.add(ELU())
model.add(Flatten())
model.add(Dropout(.2))
model.add(Dense(1024,kernel_regularizer=regularizers.l2(0.01)))
model.add(ELU())
model.add(Dense(512,kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(64,kernel_regularizer=regularizers.l2(0.01)))
model.add(ELU())
model.add(Dense(64,kernel_regularizer=regularizers.l2(0.01)))
model.add(ELU())
model.add(Dense(1))


In [21]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda, Flatten, Dropout
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.layers.advanced_activations import ELU

model = Sequential()
model.add(Cropping2D(cropping=((60,30),(0,0)), input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(75, 320, 3)))
model.add(Conv2D(16, (8, 8), strides=(4, 4), padding="same"))
model.add(ELU())
model.add(Conv2D(32, (5, 5), strides=(2, 2), padding="same"))
model.add(ELU())
model.add(Conv2D(64, (5, 5), strides=(2, 2), padding="same"))
model.add(Flatten())
model.add(Dropout(.2))
model.add(ELU())
model.add(Dense(512))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(1))

In [40]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(), loss='mse', metrics=[ 'mse'])

In [41]:
from keras.callbacks import ModelCheckpoint, Callback
checkpoint_callback = ModelCheckpoint('model{epoch:02d}.h5')
history = model.fit_generator(train_generator, steps_per_epoch=train_data_size/batch_size, \
                   validation_data=validation_generator, validation_steps=validation_data_size/batch_size, \
                   epochs=epochs, max_q_size=1, callbacks=[checkpoint_callback])

Epoch 1/80
215/214 [==============================] - 40s - loss: 1.9364 - mean_squared_error: 0.0392 - val_loss: 0.0667 - val_mean_squared_error: 0.0281
Epoch 2/80
215/214 [==============================] - 40s - loss: 0.0439 - mean_squared_error: 0.0289 - val_loss: 0.0347 - val_mean_squared_error: 0.0265
Epoch 3/80
215/214 [==============================] - 40s - loss: 0.0349 - mean_squared_error: 0.0276 - val_loss: 0.0310 - val_mean_squared_error: 0.0242
Epoch 4/80
215/214 [==============================] - 40s - loss: 0.0335 - mean_squared_error: 0.0262 - val_loss: 0.0300 - val_mean_squared_error: 0.0227
Epoch 5/80
215/214 [==============================] - 40s - loss: 0.0337 - mean_squared_error: 0.0262 - val_loss: 0.0340 - val_mean_squared_error: 0.0247
Epoch 6/80
215/214 [==============================] - 40s - loss: 0.0330 - mean_squared_error: 0.0255 - val_loss: 0.0272 - val_mean_squared_error: 0.0212
Epoch 7/80
215/214 [==============================] - 40s - loss: 0.0324 - m

215/214 [==============================] - 40s - loss: 0.2377 - mean_squared_error: 0.0309 - val_loss: 0.2162 - val_mean_squared_error: 0.0230
Epoch 54/80
215/214 [==============================] - 40s - loss: 0.2112 - mean_squared_error: 0.0302 - val_loss: 0.1977 - val_mean_squared_error: 0.0287
Epoch 55/80
215/214 [==============================] - 40s - loss: 0.1880 - mean_squared_error: 0.0296 - val_loss: 0.1730 - val_mean_squared_error: 0.0246
Epoch 56/80
215/214 [==============================] - 39s - loss: 0.1704 - mean_squared_error: 0.0311 - val_loss: 0.1519 - val_mean_squared_error: 0.0218
Epoch 57/80
215/214 [==============================] - 40s - loss: 0.1509 - mean_squared_error: 0.0290 - val_loss: 0.1495 - val_mean_squared_error: 0.0354
Epoch 58/80
215/214 [==============================] - 40s - loss: 0.1367 - mean_squared_error: 0.0294 - val_loss: 0.1230 - val_mean_squared_error: 0.0229
Epoch 59/80
215/214 [==============================] - 40s - loss: 0.1221 - mean_s

### Evaluate

In [42]:
metrics = model.evaluate_generator(test_generator, steps=test_data_size/batch_size)

In [28]:
for i in zip(model.metrics_names, metrics):
    print(i)

('loss', 0.026621897577320877)
('mean_squared_error', 0.026621897577320877)


In [43]:
for i in zip(model.metrics_names, metrics):
    print(i)

('loss', 0.02999234924169436)
('mean_squared_error', 0.018535823688829597)


In [44]:
model.save("model_3.h5")

In [ ]:
from keras.models import load_model

model = load_model("model.h5")